In [24]:
import pandas as pd

In [25]:
train_df = pd.read_csv('C:/project/physionet2012/padded_train_data_remove_tar_gen.csv', encoding='cp949')
valid_df = pd.read_csv('C:/project/physionet2012/padded_valid_data_remove_tar_gen.csv', encoding='cp949')
test_df = pd.read_csv('C:/project/physionet2012/padded_test_data_remove_tar_gen.csv', encoding='cp949')

In [26]:
def timetosecond(df):
    # 일자와 시분초 구분
    splited_time = []
    for i in df['rec_time']:
        time = i.split(',')         # 1day와 시간 구분
        time = list(reversed(time)) # 1day가 뒤로 가게 변경
        splited_time.append(time)
    # 초 단위로 치환
    second_list = []
    for each in splited_time:
        int_list = []
        
        if len(each) == 2:
            time_split = each[0].split(':')    #time_split = [' 0', '14', '00']
            for i in time_split:            # int로 변환
                element = int(i)
                int_list.append(element) # int_list = [ 0, 14, 00]
            int_list[0] = int_list[0]+24

        else:
            time_split = each[0].split(':')
            for i in time_split:        # int로 변환
                element = int(i)
                int_list.append(element)

        h_to_m = int_list[0]*60
        m = int_list[1] + h_to_m
        second = m*60 + int_list[2]
        second_list.append(second)
    # 초단위를 rec_time에 넣기
    rec_second = pd.DataFrame(second_list, columns={'rec_second'})
    df['rec_time'] = rec_second
    # 피드순, 시간 순 정렬
    df.sort_values(by=['pid','rec_time'])
        
    return df

In [27]:
train_df = timetosecond(train_df)
valid_df = timetosecond(valid_df)
test_df = timetosecond(test_df)


In [28]:
from sklearn.preprocessing import MinMaxScaler

#원핫인코딩 및 스케일링
def onehotandscale(df):
# 원핫인코딩 및 스케일링을 위한 분리
    
    # 젠더와 ICUType 원핫인코딩
    icutype = pd.get_dummies(df['ICUType'])
    gender = pd.get_dummies(df['Gender'])
    # 원핫인코딩에 컬럼 입력
    icutype.columns = ['ICU-nan','ICU-1', 'ICU-2', 'ICU-3', 'ICU-4']
    gender.columns = ['male', 'female']
    # 피드 생성
    pid = df['pid']
    
# 스케일링
    df = df.drop(['pid','ICUType','Gender'], axis = 1)
    mm = MinMaxScaler()
    mm.fit(df)
    scaled_df = mm.transform(df)
    scaled_df = pd.DataFrame(scaled_df)
    scaled_df.columns = df.columns
#원핫과 스케일 데이터프레임 병합
    df = pd.concat([pid,icutype,gender,scaled_df], axis=1)
    
    return df



In [29]:
train_df = onehotandscale(train_df)
valid_df = onehotandscale(valid_df)
test_df = onehotandscale(test_df)

In [32]:
train_df.to_csv('C:/project/physionet2012/preprocessed_train_data.csv', encoding='cp949',index=False)
valid_df.to_csv('C:/project/physionet2012/preprocessed_valid_data.csv', encoding='cp949',index=False)
test_df.to_csv('C:/project/physionet2012/preprocessed_test_data.csv', encoding='cp949',index=False)